In [ ]:
#... Taking from the previous script - check previous functions

In [ ]:
# Run experiments
for seed_idx, seed in enumerate(seeds):
    for exp in experiments:
        all_metrics_before = []
        all_metrics_after = []
        all_fairness_before = []
        all_fairness_after = []
        all_bias_diagnosis = []
        all_feature_importance = []  # Store feature importance for each flip
        
        # DataFrames to store results for this seed
        metrics_before_df = []
        metrics_after_df = []
        fairness_before_df = []
        fairness_after_df = []
        bias_diagnosis_df = []
        
        for flip_pct in flip_percentages:
            np.random.seed(seed)
            mydf2 = mydf.copy()
            mydf2['statusflip'] = mydf2['StatusLabel'].copy()
            mask = (mydf2['Gender'] == 'Female') & (mydf2['statusflip'] == 'Not Depressed')
            if mask.sum() > 0:
                mydf2.loc[mask, 'statusflip'] = np.random.choice(
                    ['Not Depressed', 'Depressed'],
                    size=mask.sum(),
                    p=[1 - flip_pct, flip_pct]
                )
            mydf2['statusflipbin'] = mydf2['statusflip'].map({'Not Depressed': 0, 'Depressed': 1})
            ybias = mydf2['statusflipbin']
            
            X_temp, X_test, ytrue_temp, ytrue_test, ybias_temp, ybias_test, sensitive_temp, sensitive_test = train_test_split(
                X, ytrue, ybias, sensitive_feature, test_size=0.2, random_state=seed, stratify=ytrue
            )
            X_train, X_val, ytrue_train, ytrue_val, ybias_train, ybias_val, sensitive_train, sensitive_val = train_test_split(
                X_temp, ytrue_temp, ybias_temp, sensitive_temp, test_size=0.25, random_state=seed, stratify=ytrue_temp
            )
            
            y_train = ytrue_train if exp['train'] == 'true' else ybias_train
            y_val_for_mitigation = ytrue_val if exp['train'] == 'true' else ybias_val
            y_test = ytrue_test if exp['test'] == 'true' else ybias_test
            
            ####################################################################
            # Use CatBoostClassifier with scale_pos_weight for imbalance
            gbm = CatBoostClassifier(random_seed=seed, scale_pos_weight=np.sum(y_train == 0) / np.sum(y_train == 1), verbose=0)
            gbm.fit(X_train, y_train)
            y_pred = gbm.predict(X_test)
            y_pred_proba = gbm.predict_proba(X_test)
            y_pred_mitigated = apply_mitigation(gbm, X_val, y_pred_proba, sensitive_test, y_val_for_mitigation, sensitive_val)

            ####################################################################
            # Store feature importance for this flip percentage
            importance = gbm.get_feature_importance()  # Use feature importances from CatBoostClassifier
            all_feature_importance.append(importance)
            ####################################################################

            male_mask = (sensitive_test == 'Male')
            female_mask = (sensitive_test == 'Female')
            overall_metrics = compute_metrics(y_test, y_pred, group_name='Overall')
            male_metrics = compute_metrics(y_test[male_mask], y_pred[male_mask], group_name='Male')
            female_metrics = compute_metrics(y_test[female_mask], y_pred[female_mask], group_name='Female')
            fairness = fairness_diagnosis(male_metrics, female_metrics)
            
            overall_metrics_mitigated = compute_metrics(y_test, y_pred_mitigated, group_name='Overall')
            male_metrics_mitigated = compute_metrics(y_test[male_mask], y_pred_mitigated[male_mask], group_name='Male')
            female_metrics_mitigated = compute_metrics(y_test[female_mask], y_pred_mitigated[female_mask], group_name='Female')
            fairness_mitigated = fairness_diagnosis(male_metrics_mitigated, female_metrics_mitigated)
            
            bias_diag = bias_diagnosis(ytrue_test, ybias_test, sensitive_test)
            
            all_metrics_before.append({'Overall': overall_metrics, 'Male': male_metrics, 'Female': female_metrics})
            all_metrics_after.append({'Overall': overall_metrics_mitigated, 'Male': male_metrics_mitigated, 'Female': female_metrics_mitigated})
            all_fairness_before.append(fairness)
            all_fairness_after.append(fairness_mitigated)
            all_bias_diagnosis.append(bias_diag)
            
            # Store results for this seed in DataFrames
            for group in ['Male', 'Female']:
                metrics_before_df.append({
                    'Flip Percentage': flip_pct * 100,
                    'Group': group,
                    **{metric: all_metrics_before[-1][group][metric] for metric in metric_names}
                })
                metrics_after_df.append({
                    'Flip Percentage': flip_pct * 100,
                    'Group': group,
                    **{metric: all_metrics_after[-1][group][metric] for metric in metric_names}
                })
            fairness_before_df.append({
                'Flip Percentage': flip_pct * 100,
                **{row['Metric']: row['Value'] for _, row in fairness.iterrows()}
            })
            fairness_after_df.append({
                'Flip Percentage': flip_pct * 100,
                **{row['Metric']: row['Value'] for _, row in fairness_mitigated.iterrows()}
            })
            bias_diagnosis_df.append({
                'Flip Percentage': flip_pct * 100,
                'Male Error Rate': bias_diag['Male'],
                'Female Error Rate': bias_diag['Female']
            })
        
        all_experiment_results[exp['name']]['metrics_before'][seed_idx] = all_metrics_before
        all_experiment_results[exp['name']]['metrics_after'][seed_idx] = all_metrics_after
        all_experiment_results[exp['name']]['fairness_before'][seed_idx] = all_fairness_before
        all_experiment_results[exp['name']]['fairness_after'][seed_idx] = all_fairness_after
        all_experiment_results[exp['name']]['bias_diagnosis'][seed_idx] = all_bias_diagnosis
        all_experiment_results[exp['name']]['feature_importance'][seed_idx] = all_feature_importance
        
        # Save seed-specific results to CSVs
        pd.DataFrame(metrics_before_df).to_csv(f"metrics_before_{exp['name']}_seed_{seed}.csv", index=False)
        pd.DataFrame(metrics_after_df).to_csv(f"metrics_after_{exp['name']}_seed_{seed}.csv", index=False)
        pd.DataFrame(fairness_before_df).to_csv(f"fairness_before_{exp['name']}_seed_{seed}.csv", index=False)
        pd.DataFrame(fairness_after_df).to_csv(f"fairness_after_{exp['name']}_seed_{seed}.csv", index=False)
        pd.DataFrame(bias_diagnosis_df).to_csv(f"bias_diagnosis_{exp['name']}_seed_{seed}.csv", index=False)

In [ ]:
# Function to calculate mean and standard error
def calculate_stats(data_list, metric, group=None):
    if group:
        values = np.array([[m[group][metric] for m in metrics] for metrics in data_list])
    else:
        values = np.array([[df[df['Metric'] == metric]['Value'].values[0] for df in dfs] for dfs in data_list])
    mean_values = np.mean(values, axis=0)
    se_values = sem(values, axis=0)
    return mean_values, se_values

# Generate plots, print mean and SE, and save mean/SE CSVs
for exp in experiments:
    exp_name = exp['name']
    results = all_experiment_results[exp_name]
    
    # Plot VIP across seeds for this experiment
    plot_vip_across_seeds(results['feature_importance'], flip_percentages, feature_names, exp_name)
    
    # DataFrames for mean and SE
    metrics_mean_se_df = []
    fairness_mean_se_df = []
    bias_diagnosis_mean_se_df = []
    
    # Print mean and SE for standard metrics and store in DataFrame
    for group in ['Male', 'Female']:
        for metric in metric_names:
            mean_values_before, se_values_before = calculate_stats(results['metrics_before'], metric, group)
            mean_values_after, se_values_after = calculate_stats(results['metrics_after'], metric, group)
            print(f"{exp_name} {group} {metric} - Before Mean: {mean_values_before}, SE: {se_values_before}")
            print(f"{exp_name} {group} {metric} - After Mean: {mean_values_after}, SE: {se_values_after}")
            
            for i, flip_pct in enumerate(flip_percentages):
                metrics_mean_se_df.append({
                    'Flip Percentage': flip_pct * 100,
                    'Group': group,
                    'Metric': metric,
                    'Before Mean': mean_values_before[i],
                    'Before SE': se_values_before[i],
                    'After Mean': mean_values_after[i],
                    'After SE': se_values_after[i]
                })
    
    # Plot metrics (with overall line and mean ± SE annotations)
    plot_metrics_across_flips(results['metrics_before'], [p*100 for p in flip_percentages], metric_names,
                             f"{exp_name} - Before Mitigation", f"metrics_vs_flip_{exp_name}_before.pdf")
    plot_metrics_across_flips(results['metrics_after'], [p*100 for p in flip_percentages], metric_names,
                             f"{exp_name} - After Mitigation", f"metrics_vs_flip_{exp_name}_after.pdf")
    
    # Fairness metrics plot (with mean ± SE annotations)
    plt.figure(figsize=(12, 8))
    for i, metric in enumerate(fairness_metrics):
        before_values = np.array([[df[df['Metric'] == metric]['Value'].values[0] for df in dfs] for dfs in results['fairness_before']])
        after_values = np.array([[df[df['Metric'] == metric]['Value'].values[0] for df in dfs] for dfs in results['fairness_after']])
        before_mean = np.mean(before_values, axis=0)
        after_mean = np.mean(after_values, axis=0)
        before_se = sem(before_values, axis=0)
        after_se = sem(after_values, axis=0)
        
        # Calculate overall mean and SE across all flip percentages
        overall_before_mean = np.mean(before_mean)
        overall_before_se = np.mean(before_se)
        overall_after_mean = np.mean(after_mean)
        overall_after_se = np.mean(after_se)
        
        print(f"{exp_name} Fairness {metric} - Before Mean: {before_mean}, SE: {before_se}")
        print(f"{exp_name} Fairness {metric} - After Mean: {after_mean}, SE: {after_se}")
        
        for j, flip_pct in enumerate(flip_percentages):
            fairness_mean_se_df.append({
                'Flip Percentage': flip_pct * 100,
                'Metric': metric,
                'Before Mean': before_mean[j],
                'Before SE': before_se[j],
                'After Mean': after_mean[j],
                'After SE': after_se[j]
            })
        
        plt.subplot(2, 2, i+1)
        plt.plot([p*100 for p in flip_percentages], before_mean, 'o-', color='#1f77b4', label='Before Mean')
        plt.fill_between([p*100 for p in flip_percentages], before_mean - before_se, before_mean + before_se, color='#1f77b4', alpha=0.2, label='Before Mean ± SE')
        plt.plot([p*100 for p in flip_percentages], after_mean, 'o-', color='#ff7f0e', label='After Mean')
        plt.fill_between([p*100 for p in flip_percentages], after_mean - after_se, after_mean + after_se, color='#ff7f0e', alpha=0.2, label='After Mean ± SE')
        
        # Add overall mean ± SE as text annotation
        plt.text(0.02, 0.98, f'Before: {overall_before_mean:.4f} ± {overall_before_se:.4f}\nAfter: {overall_after_mean:.4f} ± {overall_after_se:.4f}',
                 transform=plt.gca().transAxes, fontsize=8, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.xlabel('Flip Percentage')
        plt.ylabel(metric)
        plt.title(f'{metric} vs Flip Percentage')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.ylim(0.0, 1.0)  # Set y-axis limits to 0.0 to 1.0
    
    plt.suptitle(f'Fairness Metrics vs Flip Percentage ({exp_name})', fontsize=16, fontweight='bold')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    with PdfPages(f"fairness_vs_flip_{exp_name}.pdf") as pdf:
        pdf.savefig()
    plt.close()
    
    # Bias diagnosis plot (with overall line and mean ± SE annotations)
    plt.figure(figsize=(10, 6))
    male_bias = np.array([[bias['Male'] for bias in biases] for biases in results['bias_diagnosis']])
    female_bias = np.array([[bias['Female'] for bias in biases] for biases in results['bias_diagnosis']])
    male_mean = np.mean(male_bias, axis=0)
    female_mean = np.mean(female_bias, axis=0)
    male_se = sem(male_bias, axis=0)
    female_se = sem(female_bias, axis=0)
    
    # Calculate overall bias (mean of Male and Female error rates)
    overall_bias = (male_bias + female_bias) / 2
    overall_mean = np.mean(overall_bias, axis=0)
    overall_se = sem(overall_bias, axis=0)
    
    # Calculate overall mean and SE across all flip percentages
    overall_male_mean = np.mean(male_mean)
    overall_male_se = np.mean(male_se)
    overall_female_mean = np.mean(female_mean)
    overall_female_se = np.mean(female_se)
    overall_overall_mean = np.mean(overall_mean)
    overall_overall_se = np.mean(overall_se)
    
    print(f"{exp_name} Bias Diagnosis - Male Mean: {male_mean}, SE: {male_se}")
    print(f"{exp_name} Bias Diagnosis - Female Mean: {female_mean}, SE: {female_se}")
    print(f"{exp_name} Bias Diagnosis - Overall Mean: {overall_mean}, SE: {overall_se}")
    
    for i, flip_pct in enumerate(flip_percentages):
        bias_diagnosis_mean_se_df.append({
            'Flip Percentage': flip_pct * 100,
            'Male Mean': male_mean[i],
            'Male SE': male_se[i],
            'Female Mean': female_mean[i],
            'Female SE': female_se[i],
            'Overall Mean': overall_mean[i],
            'Overall SE': overall_se[i]
        })
    
    plt.plot([p*100 for p in flip_percentages], male_mean, 'o-', color='#1f77b4', label='Male Mean')
    plt.fill_between([p*100 for p in flip_percentages], male_mean - male_se, male_mean + male_se, color='#1f77b4', alpha=0.2, label='Male Mean ± SE')
    plt.plot([p*100 for p in flip_percentages], female_mean, 'o-', color='#ff7f0e', label='Female Mean')
    plt.fill_between([p*100 for p in flip_percentages], female_mean - female_se, female_mean + female_se, color='#ff7f0e', alpha=0.2, label='Female Mean ± SE')
    plt.plot([p*100 for p in flip_percentages], overall_mean, 'o-', color='#2ca02c', label='Overall Mean')
    plt.fill_between([p*100 for p in flip_percentages], overall_mean - overall_se, overall_mean + overall_se, color='#2ca02c', alpha=0.2, label='Overall Mean ± SE')
    
    # Add overall mean ± SE as text annotation
    plt.text(0.02, 0.98, f'Male: {overall_male_mean:.4f} ± {overall_male_se:.4f}\nFemale: {overall_female_mean:.4f} ± {overall_female_se:.4f}\nOverall: {overall_overall_mean:.4f} ± {overall_overall_se:.4f}',
             transform=plt.gca().transAxes, fontsize=8, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.xlabel('Flip Percentage')
    plt.ylabel('Bias Error Rate')
    plt.title(f'Bias Error Rate vs Flip Percentage ({exp_name})')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.ylim(0.0, 1.0)  # Set y-axis limits to 0.0 to 1.0
    plt.tight_layout()
    with PdfPages(f"bias_error_vs_flip_{exp_name}.pdf") as pdf:
        pdf.savefig()
    plt.close()
    
    # Save mean and SE results to CSVs
    pd.DataFrame(metrics_mean_se_df).to_csv(f"metrics_mean_se_{exp_name}.csv", index=False)
    pd.DataFrame(fairness_mean_se_df).to_csv(f"fairness_mean_se_{exp_name}.csv", index=False)
    pd.DataFrame(bias_diagnosis_mean_se_df).to_csv(f"bias_diagnosis_mean_se_{exp_name}.csv", index=False)